resize selenium window?

In [1]:

var rows = 6;
var columns = 6;
var screen = {
    height: 1024 * 6,
    width: 1350 * 6
};

var getScreenSize = () => {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

var resizeWindow = (i = 0) => {
    i = Math.abs(i + (typeof OFFSET !== 'undefined' ? OFFSET : 0));
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => typeof screen === 'undefined' ? getScreenSize() : screen)
        .catch(e => console.log(e))
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .catch(e => console.log(e))
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}

if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


ReferenceError: client is not defined

only one window?



In [2]:
var importer = require('../Core');

var closeAllTabs = (keep) => {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var closeAllWindows = (keep) => {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

var onlyOneWindow = () => {
    return client
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .catch(e => console.log(e))
    // TODO: close all tabs
}

module.exports = onlyOneWindow;



[Function: onlyOneWindow]

tile selenium chrome windows?



In [3]:
var importer = require('../Core');
var {getAllSessionUrls} = importer.import(['resize selenium window', 'get all session urls'], {client})

var openUrl = (url) => {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
}

var createNewWindows = (urls) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var tileWindows = (urls) => {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .then(() => getAllSessionUrls())
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



ReferenceError: client is not defined

In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell('tile chrome windows')
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
[22:44:47]  COMMAND	GET 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c/window_handle"
[22:45:06]  COMMAND	POST 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c/window"
[22:45:09]  COMMAND	GET 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c"
[22:45:10]  COMMAND	GET 	 "/wd/hub/session/07e2149c2d8f2e9b1402ae2918ec0637/window_handle"
[22:45:10]  COMMAND	POST 	 "/wd/hub/session/07e2149c2d8f2e9b1402ae2918ec0637/window"
[22:45:10]  COMMAND	GET 	 "/wd/hub/session/07e2149c2d8f2e9b1402ae2918ec0637"
[22:45:10]  COMMAND	GET 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9/window_handle"
[22:45:10]  COMMAND	POST 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9/window"
[22:45:10]  COMMAND	GET 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9"
[22:45:10]  COMMAND	GET 	 "/wd/hub/session/8a6e9afbfb2e672c5899a78ec6e4b192/window_handle"
[22:45:10]  COMMAND	POST 	 "/wd/hub/session/8a6e9afbfb2e672c5899a78ec6e4b192/window"
[22:45:10]  COMMAND	GET 	 "/w

get all session and window urls?



In [ ]:
var importer = require('../Core');

var getAllSessionUrls = (reposition = true) => {
    var windowCounter = -1;
    var session = client.requestHandler.sessionID;
    return client
        .getActiveSessions()
        .then(r => {
            console.log(r);
            return importer.runAllPromises(r.map(s => resolve => {
                client.requestHandler.sessionID = s;
                return client
                    .getUrl()
                    .windowHandles()
                    .then(h => importer.runAllPromises(h.value.map(hwnd => resolve => {
                        windowCounter++
                        return client
                            .window(hwnd)
                            .then(() => reposition ? resizeWindow(windowCounter) : [])
                            .getUrl()
                            .then(r => resolve(r))
                            .catch(e => console.log(e))
                    })))
                    .then(r => resolve(r))
                    .catch(e => console.log(e))
            }))
        })
        .then(r => {
            client.requestHandler.sessionID = session;
            return r;
        })
        .catch(e => console.log(e))
}

module.exports = getAllSessionUrls;


In [5]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell(['resize selenium window', 'get all session urls'], false)
    .then(r => r.getAllSessionUrls(false))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[23:03:13]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window_handle"
[23:03:13]  COMMAND	POST 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window"
[23:03:14]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5"
[ 'c5d92fbc3c737c3c0c06b3a223a65cb5' ]
[23:03:14]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5"
c5d92fbc3c737c3c0c06b3a223a65cb5
[23:03:14]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window_handle"
[23:03:14]  COMMAND	POST 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window"
[23:03:14]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5"
[23:03:14]  COMMAND	GET 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c/window_handle"
[23:03:15]  COMMAND	POST 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c/window"
[23:03:15]  COMMAND	GET 	 "/wd/hub/session/2ea23b58bb2bcb06dde32598cab22a7c"
[23:03:15]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737

[ 'https://www.facebook.com/messages/t/ragunr',
  'https://www.facebook.com/messages/t/johan.larson.56',
  'https://www.facebook.com/messages/t/abe.pralle',
  'https://www.facebook.com/messages/t/wolfdieter.otte',
  'https://www.facebook.com/messages/t/carisa.knight6000',
  'https://www.facebook.com/messages/t/talbert.tso',
  'https://www.facebook.com/messages',
  'https://www.facebook.com/messages/t/devin.west2',
  'https://www.facebook.com/messages/t/eurostar88',
  'https://www.facebook.com/messages/t/robert.chandler',
  'https://www.facebook.com/messages/t/aneesh.karve',
  'https://www.facebook.com/messages',
  'https://www.facebook.com/messages/t/jaimeneufer',
  'https://www.facebook.com/messages/t/robert.holweger',
  'https://www.facebook.com/messages/t/keifer.street',
  'https://www.facebook.com/Ross.Sephton/posts/10208647032226634' ]

Monitor many chats at once and continuously


In [1]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
var tileWindows;
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell(['tile chrome windows', 'log in facebook'])
    .then(r => {
        tileWindows = r.tileWindows;
        return r.loginFacebook()
    })
    .then(() => tileWindows([
        'https://www.facebook.com/messages/t/ragunr',
        'https://www.facebook.com/messages/t/johan.larson.56',
        'https://www.facebook.com/messages/t/abe.pralle',
        'https://www.facebook.com/messages/t/wolfdieter.otte',
        'https://www.facebook.com/messages/t/carisa.knight6000',
        'https://www.facebook.com/messages/t/talbert.tso',
        'https://www.facebook.com/messages/t/abe.anwary',
        'https://www.facebook.com/messages/t/devin.west2',
        'https://www.facebook.com/messages/t/eurostar88',
        'https://www.facebook.com/messages/t/robert.chandler',
        'https://www.facebook.com/messages/t/aneesh.karve',
        'https://www.facebook.com/messages/t/abe.anwary',
        'https://www.facebook.com/messages/t/jaimeneufer',
        'https://www.facebook.com/messages/t/robert.holweger',
        'https://www.facebook.com/messages/t/keifer.street',
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[22:49:40]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window_handle"
[22:50:09]  COMMAND	POST 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5/window"
[22:50:09]  COMMAND	GET 	 "/wd/hub/session/c5d92fbc3c737c3c0c06b3a223a65cb5"
[22:50:09]  COMMAND	GET 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9/window_handle"
[22:50:09]  COMMAND	POST 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9/window"
[22:50:09]  COMMAND	GET 	 "/wd/hub/session/e1c92786de9facc9d3bb90bdc20e0ab9"
[22:50:09]  COMMAND	GET 	 "/wd/hub/session/8a6e9afbfb2e672c5899a78ec6e4b192/window_handle"
[22:50:09]  COMMAND	POST 	 "/wd/hub/session/8a6e9afbfb2e672c5899a78ec6e4b192/window"
[22:50:10]  COMMAND	GET 	 "/wd/hub/session/8a6e9afbfb2e672c5899a78ec6e4b192"
[22:50:10]  COMMAND	GET 	 "/wd/hub/session/07e2149c2d8f2e9b1402ae2918ec0637/window_handle"
[22:50:10]  COMMAND	POST 	 "/wd/hub/session/07e2149c2d8f2e9b1402ae2918ec0637/window"
[22:50:10]  COMMAND	GET 	 "/w

{ sessionId: 'c5d92fbc3c737c3c0c06b3a223a65cb5',
  status: 0,
  value: 
   [ 'CDwindow-60cd5b23-15fb-42cb-bc52-c35d16945587',
     'CDwindow-46c50a65-703c-47db-ba6d-840705ec68cf',
     'CDwindow-bdb33ef4-db2f-42d4-8871-f3da8d6e8634',
     'CDwindow-d2561ce5-3c71-4de0-b728-ee2a62ab2e80',
     'CDwindow-b0f1f382-87cd-43c9-9ca1-657c6f94fc0c',
     'CDwindow-139a86f8-9ec1-4b0d-8510-7ea17ac237fa',
     'CDwindow-b8b3d6be-3816-480f-b14e-aad0a1c0a4a0',
     'CDwindow-edf5fb69-1b2e-4ed8-ab5f-3b6babc78e71',
     'CDwindow-def209d4-b87a-4a6f-b204-8d8a142a86b4',
     'CDwindow-d9afdbf9-adc3-49a4-9e32-4b80c438ebdc',
     'CDwindow-b0095cac-d439-41af-8242-5a3f4db1ff79',
     'CDwindow-80889e87-7b55-4d8b-89cf-3675d74cb752',
     'CDwindow-f6528342-1608-49b1-b06d-44077205a3e6',
     'CDwindow-f5d6132d-7dcb-4453-9303-d4aa6746ed6a',
     'CDwindow-fc1bfa4a-217f-42a7-9f12-d09a79392cb0' ] }

clean up old selenium sessions?

In [5]:
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
var runSeleniumCell = importer.import('selenium cell');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

$$.async();
execCmd('docker logs act-selenium')
    .then(r => regexToArray(/\/session\/(.+?)\//ig, r.join('\n'), 1))
    .then(r => r.filter((s, i, arr) => arr.indexOf(s) === i))
    .then(() => {
        return runSeleniumCell();
    })
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r))



Waiting xvfb...
Using installed websockify at /usr/bin/websockify
Starting webserver and WebSockets proxy on port 6080



sed: can't read /usr/profile/Default/Preferences: No such file or directory
sed: can't read /usr/profile/Default/Preferences: No such file or directory
Failed to read: session.ignoreBorder
Setting default value
Failed to read: session.forcePseudoTransparency
Setting default value
Failed to read: session.colorsPerChannel
Setting default value
Failed to read: session.doubleClickInterval
Setting default value
Failed to read: session.tabPadding
Setting default value
Failed to read: session.styleOverlay
Setting default value
Failed to read: session.slitlistFile
Setting default value
Failed to read: session.appsFile
Setting default value
Failed to read: session.tabsAttachArea
Setting default value
Failed to read: session.cacheLife
Setting default value
Failed to read: session.cacheMax
Setting default value
Failed to read: session.autoRaiseDelay
Setting default value
Failed to read: session.ignoreBorder
Setting default value
Failed to read: session.forcePseudoTransparency
Setting default valu

04:29:03.689 INFO - Selenium build info: version: '3.5.0', revision: '8def36e068'
04:29:03.691 INFO - Launching a standalone Selenium Server
2017-11-10 04:29:03.734:INFO::main: Logging initialized @837ms to org.seleniumhq.jetty9.util.log.StdErrLog
04:29:03.836 INFO - Driver class not found: com.opera.core.systems.OperaDriver


Navigate to this URL:

    http://812c908edbad:6080/vnc.html?host=812c908edbad&port=6080

Press Ctrl-C to exit


04:29:03.894 INFO - Driver provider class org.openqa.selenium.ie.InternetExplorerDriver registration is skipped:
 registration capabilities Capabilities [{ensureCleanSession=true, browserName=internet explorer, version=, platform=WINDOWS}] does not match the current platform LINUX
04:29:03.894 INFO - Driver provider class org.openqa.selenium.edge.EdgeDriver registration is skipped:
 registration capabilities Capabilities [{browserName=MicrosoftEdge, version=, platform=WINDOWS}] does not match the current platform LINUX
04:29:03.895 INFO - Driver provid

10/11/2017 04:29:04   repaint the screen, also see the -fixscreen option for
10/11/2017 04:29:04   periodic repaints.
10/11/2017 04:29:04   Note: '-scale' is on and this can cause more problems.
10/11/2017 04:29:04 
10/11/2017 04:29:04 XFIXES available on display, resetting cursor mode
10/11/2017 04:29:04   to: '-cursor most'.
10/11/2017 04:29:04   to disable this behavior use: '-cursor arrow'
10/11/2017 04:29:04   or '-noxfixes'.
10/11/2017 04:29:04 using XFIXES for cursor drawing.
10/11/2017 04:29:04 GrabServer control via XTEST.
10/11/2017 04:29:04 
10/11/2017 04:29:04 Scroll Detection: -scrollcopyrect mode is in effect to
10/11/2017 04:29:04   use RECORD extension to try to detect scrolling windows
10/11/2017 04:29:04   (induced by either user keystroke or mouse input).
10/11/2017 04:29:04   If this yields undesired behavior (poor response, painting
10/11/2017 04:29:04   errors, etc) it may be disabled via: '-noscr'
10/11/2017 04:29:04   Also see the -help entry for tuning paramete

2017-11-10 04:48:00.019:INFO:osjshC.ROOT:qtp971848845-10: org.openqa.selenium.remote.server.WebDriverServlet-4678c730: Initialising WebDriverServlet
04:48:00.061 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@3cf6ad4f
04:48:00.063 INFO - /session/88484f1e0aa90e53f7c2c2d315a14401/window_handle: Executing GET on /session/88484f1e0aa90e53f7c2c2d315a14401/window_handle (handler: NoSessionHandler)
04:48:00.100 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@227d2c21

04:48:00.100 INFO - /session/5096e1c4d3d1e7e87910ef660b02b898/window_handle: Executing GET on /session/5096e1c4d3d1e7e87910ef660b02b898/window_handle (handler: NoSessionHandler)
04:48:00.111 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@3946e62b
04:48:00.112 INFO - /session/14e96631161d2e2f1a86c86375095951/window_handle: Executing GET on /session/14e96631161d2e2f1a86c86375095951/window_handle (handler: NoSe

{}